In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Load the dataset
df = pd.read_csv("tips.csv")

# features and target
X = df[['total_bill', 'gender', 'smoker', 'day', 'time', 'size']]
y = df[['tip']]

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=['gender', 'smoker', 'day', 'time'])

# Standardize numerical features
scaler_X = StandardScaler()
X_encoded[['total_bill', 'size']] = scaler_X.fit_transform(X_encoded[['total_bill', 'size']])

# Standardize the target
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

# Convert to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((
    X_encoded.values.astype(np.float32),
    y_scaled.astype(np.float32)
))

# Shuffle and split into train and test datasets
dataset = dataset.shuffle(buffer_size=len(X_encoded), seed=42)
train_size = int(0.8 * len(X_encoded))
train_dataset = dataset.take(train_size).batch(32)
test_dataset = dataset.skip(train_size).batch(32)

# Build the neural network model
model = Sequential([
    Dense(16, activation='relu', input_shape=(X_encoded.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model using Adam optimizer
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(train_dataset, epochs=25, validation_data=test_dataset, verbose=1)

# Predict on test dataset
y_pred_scaled = model.predict(test_dataset)
y_true_scaled = np.concatenate([y.numpy() for _, y in test_dataset], axis=0)

# Inverse transform predictions and true values
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_true_scaled)

# Calculate and print Mean Absolute Error
mae = mean_absolute_error(y_true, y_pred)
print(f"Test Mean Absolute Error (unscaled): {mae:.2f}")


Epoch 1/25


C:\Users\atade\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.2504 - mae: 0.8939 - val_loss: 1.0619 - val_mae: 0.8153
Epoch 2/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1466 - mae: 0.8559 - val_loss: 0.8475 - val_mae: 0.7474
Epoch 3/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1684 - mae: 0.8718 - val_loss: 1.0759 - val_mae: 0.8014
Epoch 4/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0003 - mae: 0.7913 - val_loss: 0.9226 - val_mae: 0.7729
Epoch 5/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0568 - mae: 0.7974 - val_loss: 1.0020 - val_mae: 0.8239
Epoch 6/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9637 - mae: 0.7577 - val_loss: 0.9560 - val_mae: 0.7675
Epoch 7/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0006 - mae: 0.7923 - val_loss: 0.7449 - val_mae: 0.6686
Epoch 8/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9167 - mae: 0.7511 - val_loss: 0.8131 - val_mae: 0.7191
Epoch 9/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9387 - mae: 0.7739 - v